# ERA5 correction 
Adapted from Eric Gagliano's notebook here: https://github.com/gbrencher/AMATH563-InSAR-denoiser/blob/main/data-processing/era5_correction_individual_igrams.ipynb

Note: This code will only work if in the MintPy source code (tropo_pyaps3.py) we comment out lines 657 (ref_y, ref_x = int(atr['REF_Y']), int(atr['REF_X'])) and 666 (data -= data[ref_y, ref_x]). This code subtracts out the value at the reference point. Since we haven't set a reference point in the metadata of these interferograms, it won't work. We can handle this later

In [1]:
import mintpy
from pathlib import Path
from dateutil.parser import parse as parse_date
from pathlib import Path
from typing import List, Union
from osgeo import gdal
from pathlib import Path
from typing import List, Union
import os
import glob
import matplotlib.pyplot as plt
from mintpy.utils import ptime, readfile, writefile, utils as ut

In [2]:
orbit_list = ['AT137']
year_list = ['2017', '2018', '2019', '2020', '2021']

In [3]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [4]:
def correct_era5(orbit_list, year_list, weather_dir = '/tmp'):
    home_path_d = '/mnt/d/indennt'
    home_path = '/mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data'
    for orbit in orbit_list:
        for year in year_list:
            print(f'working on {orbit}, {year}')
            data_path = f'{home_path_d}/hyp3_app/{orbit}/{year}'
            hyp3_list = os.listdir(data_path)
            for i, granule in enumerate(hyp3_list):
                print(f'working on {i}/{len(hyp3_list)}')
                granule_path = f'{data_path}/{granule}'
                mintpy_path = f'{home_path}/signal_mintpy/{orbit}/{granule}'

                if os.path.exists(f'{granule_path}/{granule}_unw_phase_ERA5.tif'):
                    print(f'ERA5 already in {granule}, skipping') 
                    continue

                # make output dir for mintpy
                if not os.path.exists(mintpy_path):
                    os.mkdir(mintpy_path)
                
                os.chdir(mintpy_path)

                CONFIG_TXT=f"""
                mintpy.load.processor        = hyp3
                ##---------interferogram datasets:
                mintpy.load.unwFile          = {granule_path}/*_unw_phase.tif
                mintpy.load.corFile          = {granule_path}/*_corr.tif
                ##---------geometry datasets:
                mintpy.load.demFile          = {granule_path}/*_dem.tif
                mintpy.load.incAngleFile     = {granule_path}/*_lv_theta.tif
                mintpy.load.azAngleFile      = {granule_path}/*_lv_phi.tif
                mintpy.load.waterMaskFile    = {granule_path}/*_water_mask.tif
                """
                mintpy_config = f'{mintpy_path}/{year}_Sen{orbit}.txt'
                write_config_file(mintpy_config, CONFIG_TXT)

                !smallbaselineApp.py {mintpy_config} --dostep load_data
                geom_path = 'inputs/geometryGeo.h5'
            
                unw_filename = f'{granule}_unw_phase.unw'
            
                data, atr = readfile.read(f'{granule_path}/{granule}_unw_phase.tif', datasetName='phase')
                atr['FILE_TYPE'] = '.unw'
                writefile.write({'phase':data},unw_filename,metadata=atr)
                
                !tropo_pyaps3.py -f $unw_filename -g $geom_path --weather-dir $weather_dir
                unw_filename_corr = f'{unw_filename[:-4]}_ERA5.unw'
                unw_filename_out = f'{unw_filename[:-4]}_ERA5.tif'
                !gdal_translate -of GTiff -b 2 -a_srs EPSG:32612 $unw_filename_corr $unw_filename_out
                !cp *_ERA5.tif $granule_path && rm -R $mintpy_path

In [5]:
correct_era5(orbit_list, year_list)

working on AT137, 2017
working on 0/45
ERA5 already in S1BB_20170613T011036_20170625T011036_VVP012_INT40_G_ueF_0FFD, skipping
working on 1/45
ERA5 already in S1BB_20170613T011036_20170707T011037_VVP024_INT40_G_ueF_BDAA, skipping
working on 2/45
ERA5 already in S1BB_20170613T011036_20170731T011038_VVP048_INT40_G_ueF_52B1, skipping
working on 3/45
ERA5 already in S1BB_20170613T011036_20170812T011039_VVP060_INT40_G_ueF_187A, skipping
working on 4/45
ERA5 already in S1BB_20170613T011036_20170824T011040_VVP072_INT40_G_ueF_A6C1, skipping
working on 5/45
ERA5 already in S1BB_20170613T011036_20170905T011040_VVP084_INT40_G_ueF_8684, skipping
working on 6/45
ERA5 already in S1BB_20170613T011036_20170917T011041_VVP096_INT40_G_ueF_A773, skipping
working on 7/45
ERA5 already in S1BB_20170613T011036_20170929T011041_VVP108_INT40_G_ueF_A0CA, skipping
working on 8/45
ERA5 already in S1BB_20170613T011036_20171011T011041_VVP120_INT40_G_ueF_708A, skipping
working on 9/45
ERA5 already in S1BB_20170625T0110

/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


write file: S1BB_20190709T011050_20190802T011051_VVP024_INT40_G_ueF_F909_unw_phase.unw
write file: S1BB_20190709T011050_20190802T011051_VVP024_INT40_G_ueF_F909_unw_phase.unw.rsc
weather model: ERA5 - dry (hydrostatic) and wet delay
weather directory: /tmp
output tropospheric delay     time-series file: inputs/ERA5.h5
output corrected displacement time-series file: S1BB_20190709T011050_20190802T011051_VVP024_INT40_G_ueF_F909_unw_phase_ERA5.unw
read dates/time info from file: S1BB_20190709T011050_20190802T011051_VVP024_INT40_G_ueF_F909_unw_phase.unw
time of cloest available product: 01:00 UTC
/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(

--------------------------------------------------------------------------------
Download global atmospheric model files...
update mode: ON
out

## set reference point
switch to env with xarray

In [1]:
import os
import numpy as np
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import rioxarray
import seaborn as sns
import geopandas as gpd
from glob import glob

In [2]:
# load in single igram and other data 
def hyp3_to_xarray_single(path):
    '''
    Reads hyp3 outputs into xarray dataset from single hyp3 folder 
    '''
    # globs for data to load
    unw_phase_path = glob(f'{path}/*unw_phase.tif')[0]
    era5_path = glob(f'{path}/*ERA5.tif')[0]
    meta_path = glob(f'{path}/S1*.txt')[0]

    # list granules for coordinate
    granule = os.path.split(unw_phase_path)[-1][0:-14]

    d = {}
    with open(meta_path) as f:
        for line in f:
            (key, val) = line.split(':')
            d[key] = str.strip(val)

    # read unw_phase into data array and assign coordinates
    da = xr.open_dataset(unw_phase_path)
    da = da.assign_coords({'granule':('granule', [granule])})
    for item in d.keys():
            da = da.assign_coords({item:('granule', [d[item]])})
    
    # concatenate into dataset and rename variable
    ds = da.rename({'band_data': 'unw_phase'})

    #open coherence and dem into datasets
    era5_ds = xr.open_dataset(era5_path)

    # add coherence and dem to unw_phase dataset
    ds['era5_phase'] = (('band', 'y', 'x'), era5_ds.band_data.values)

    # remove band coordinate
    ds = ds.squeeze()

    return ds

In [7]:
def set_era5_ref(orbit, year_list):
    home_path = '/mnt/d/indennt'
    for orbit in orbit_list:
        for year in year_list:
            data_path = f'{home_path}/hyp3_app/{orbit}/{year}'
            hyp3_list = os.listdir(data_path)
            for i, granule in enumerate(hyp3_list):
                print(f'working on {orbit}, {year}, {i}/{len(hyp3_list)}')
                granule_path = f'{data_path}/{granule}'
                os.chdir(granule_path)
                
                # if readme files are still present
                !rm *.md*

                ds = hyp3_to_xarray_single(granule_path)
                ref_value = ds.era5_phase.sel(y=ds['Y coordinate of the reference point in the map projection'],
                              x=ds['X coordinate of the reference point in the map projection'],
                              method='nearest').item()
                ds['era5_phase'] = ds['era5_phase'] - ref_value
                ds.era5_phase.rio.to_raster(f'{granule_path}/{granule}_unw_phase_ERA5.tif')

In [4]:
orbit_list = ['AT137']
year_list = ['2017', '2018', '2019', '2020', '2021']

In [8]:
set_era5_ref(orbit_list, year_list)

working on AT137, 2017, 0/45
working on AT137, 2017, 1/45
working on AT137, 2017, 2/45
working on AT137, 2017, 3/45
working on AT137, 2017, 4/45
working on AT137, 2017, 5/45
working on AT137, 2017, 6/45
working on AT137, 2017, 7/45
working on AT137, 2017, 8/45
working on AT137, 2017, 9/45
working on AT137, 2017, 10/45
working on AT137, 2017, 11/45
working on AT137, 2017, 12/45
working on AT137, 2017, 13/45
working on AT137, 2017, 14/45
working on AT137, 2017, 15/45
working on AT137, 2017, 16/45
working on AT137, 2017, 17/45
working on AT137, 2017, 18/45
working on AT137, 2017, 19/45
working on AT137, 2017, 20/45
working on AT137, 2017, 21/45
working on AT137, 2017, 22/45
working on AT137, 2017, 23/45
working on AT137, 2017, 24/45
working on AT137, 2017, 25/45
working on AT137, 2017, 26/45
working on AT137, 2017, 27/45
working on AT137, 2017, 28/45
working on AT137, 2017, 29/45
working on AT137, 2017, 30/45
working on AT137, 2017, 31/45
working on AT137, 2017, 32/45
working on AT137, 20